In [1]:
import sys
sys.path.append('../../code/')
from utils.data_io import read_json
from Ques_pair import Ques_pair, Type

In [2]:
ref_file = read_json("../../datasets/qald9plus/qald_9_pp_test_wikidata.json")
pred_file = read_json("../../pred_files/qald9-plus-all/en.json")

In [3]:
ref_qalds = [ref for ref in ref_file["questions"]]
pred_qalds = [pred for pred in pred_file["questions"]]

In [4]:
n = 5
for ref, pred in zip(ref_qalds[:n], pred_qalds[:n]):
    ques_pair = Ques_pair(ref, pred)
    ques_pair.print(print_answer=True)
    print()

id: 99
question: What is the time zone of Salt Lake City?
ref sparql: SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q23337>  <http://www.wikidata.org/prop/direct/P421>  ?o1 .  }
pred sparql: SELECT DISTINCT  ?uri WHERE  {  wd:Q23434 wdt:P421  ?uri .  }  
ref_answer:
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3134980'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q2212'}}]
pred_answer:
 []


id: 98
question: Who killed Caesar?
ref sparql: SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q1048>  <http://www.wikidata.org/prop/direct/P157>  ?o1 .  }
pred sparql: SELECT DISTINCT  ?uri WHERE  {  wd:Q1471987 wdt:P157  ?uri .  }  
ref_answer:
 [{'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q172248'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q207370'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q294846'}}, {'o1': {'type': 'uri', 'value': 'http://www.wikidata.org/

In [5]:
entity_f1_sum = 0
relation_f1_sum = 0
num_of_ques = len(ref_qalds)

for ref, pred in zip(ref_qalds, pred_qalds):
    ques_pair = Ques_pair(ref, pred)
    entity_f1_sum += ques_pair.calculate_entity_f1()
    relation_f1_sum += ques_pair.calculate_relation_f1()

macro_entity_f1 = entity_f1_sum / num_of_ques
macro_relation_f1 = relation_f1_sum / num_of_ques

In [6]:
macro_entity_f1

0.0058823529411764705

In [7]:
macro_relation_f1

0.015416666666666667

In [8]:
# check reason of no answer
counter = 0

for ref, pred in zip(ref_qalds, pred_qalds):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.pred_answer==[]:
        ques_pair.print()
        counter += 1

print(str(counter) + " SPARQL queries don't give an answer")

# counter = sum([ques_pair.print() or 1 for ref, pred in zip(ref_ques_list, pred_ques_list) if (ques_pair := Ques_pair(ref, pred)).pred_answer == []])
# print(counter)


id: 99
question: What is the time zone of Salt Lake City?
ref sparql: SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q23337>  <http://www.wikidata.org/prop/direct/P421>  ?o1 .  }
pred sparql: SELECT DISTINCT  ?uri WHERE  {  wd:Q23434 wdt:P421  ?uri .  }  

id: 98
question: Who killed Caesar?
ref sparql: SELECT DISTINCT ?o1 WHERE { <http://www.wikidata.org/entity/Q1048>  <http://www.wikidata.org/prop/direct/P157>  ?o1 .  }
pred sparql: SELECT DISTINCT  ?uri WHERE  {  wd:Q1471987 wdt:P157  ?uri .  }  

id: 84
question: Which American presidents were in office during the Vietnam War?
ref sparql: SELECT DISTINCT ?o WHERE { <http://www.wikidata.org/entity/Q8740>  <http://www.wikidata.org/prop/direct/P580>  ?startWar . <http://www.wikidata.org/entity/Q8740>  <http://www.wikidata.org/prop/direct/P582>  ?endWar . <http://www.wikidata.org/entity/Q30>  <http://www.wikidata.org/prop/P6> ?blank . ?blank <http://www.wikidata.org/prop/statement/P6> ?o . ?blank <http://www.wikidata.org/p

In [9]:
# check correct answers
counter = 0

for ref, pred in zip(ref_qalds, pred_qalds):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.pred_answer==ques_pair.ref_answer:
        ques_pair.print()
        counter += 1

print(str(counter) + " questions are 100% correct")

id: 81
question: Butch Otter is the governor of which U.S. state?
ref sparql: PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?res WHERE { ?res wdt:P6 wd:Q39593 . }
pred sparql: SELECT DISTINCT  ?uri WHERE  {  wd:Q983650 wdt:P6  ?uri .  }  

id: 168
question: Which actors play in Big Bang Theory?
ref sparql: SELECT DISTINCT ?uri WHERE { <http://www.wikidata.org/entity/Q8539> <http://www.wikidata.org/prop/direct/P161> ?uri}
pred sparql: SELECT DISTINCT  ?uri WHERE  {  wd:Q8539 wdt:P161  ?uri .  }  

id: 149
question: Which U.S. state has the highest population density?
ref sparql: PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> SELECT ?uri { ?uri wdt:P31 wd:Q35657 . ?uri wdt:P1082 ?population . ?uri wdt:P2046 ?area . BIND((?population/?area) AS ?density ) } ORDER BY DESC(?density) LIMIT 1
pred sparql: SELECT DISTINCT  ?uri WHERE  {   ?uri wdt:P31 wd:Q35657 ; wdt:P2046  ?area ; wdt:P108

In [10]:
# check ASK queries
count_all = 0
count_ask = 0

for ref, pred in zip(ref_qalds, pred_qalds):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.BOOLEAN:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "ASK" in ques_pair.pred_sparql:
            count_ask += 1

print(f'''{count_ask} of {count_all} predicted queries contain "ASK"''')

id: 117
question: Is Pamela Anderson a vegan?
ref sparql: ASK where { <http://www.wikidata.org/entity/Q83325> <http://www.wikidata.org/prop/direct/P1576> <http://www.wikidata.org/entity/Q83364>}
pred sparql: ASK WHERE  {  wd:Q40523 wdt:P140 wd:Q10908 .  }  
ref_answer:
 True
pred_answer:
 False

id: 79
question: Are there any castles in the United States?
ref sparql: PREFIX wd: <http://www.wikidata.org/entity/> PREFIX wdt: <http://www.wikidata.org/prop/direct/> ASK WHERE { { ?uri wdt:P31 wd:Q23413. } UNION { ?uri wdt:P31 ?castlesubtype . ?castlesubtype wdt:P279* wd:Q23413 } ?uri wdt:P17 wd:Q30 }
pred sparql: SELECT DISTINCT  ?uri WHERE  {   ?uri wdt:P31 wd:Q33506 .  ?uri wdt:P17 wd:Q30 .  }  
ref_answer:
 False
pred_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q259242'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q263750'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q280171'}}, {'uri': {'type': 'uri', 'value': 

In [11]:
# check COUNT queries
count_all = 0
count_count = 0

for ref, pred in zip(ref_qalds, pred_qalds):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.COUNT:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "COUNT" in ques_pair.pred_sparql:
            count_count += 1

print(f'''{count_count} of {count_all} predicted queries contain "COUNT"''')

id: 42
question: Which countries have places with more than two caves?
ref sparql: PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT DISTINCT ?uri WHERE { ?cave wdt:P31 wd:Q35509 ; wdt:P17 ?uri . } GROUP BY ?uri HAVING (COUNT(?cave) > 2)
pred sparql: SELECT DISTINCT  ?uri WHERE  {   ?uri wdt:P31 wd:Q6256 .  ?uri wdt:P17 wd:Q30 .  ?uri wdt:P2046  ?area .  }  GROUP BY  ?uri HAVING(COUNT(  ?area)>2)
ref_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q233'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q236'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q242'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q241'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q252'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34020'}}, {'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q46395'}}, {'uri': {'t

In [12]:
# check RANK queries
count_all = 0
count_contains_order = 0

for ref, pred in zip(ref_qalds, pred_qalds):
    ques_pair = Ques_pair(ref, pred)
    if ques_pair.type == Type.RANK:
        ques_pair.print(print_answer=True)
        count_all += 1
        if "ORDER" in ques_pair.pred_sparql:
            count_contains_order += 1

print(f'''{count_contains_order} of {count_all} predicted queries contain "ORDER"''')

id: 86
question: What is the highest mountain in Germany?
ref sparql: PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> SELECT ?uri WHERE { ?uri wdt:P31 wd:Q8502 ; wdt:P2044 ?elevation ; wdt:P17 wd:Q183 . } ORDER BY DESC(?elevation) LIMIT 1
pred sparql: SELECT  ?res WHERE  {   ?res wdt:P31 wd:Q8502 .  ?res wdt:P131+ wd:Q1202 .  ?res wdt:P2044  ?height .  }  ORDER BY DESC(  ?height) LIMIT 1
ref_answer:
 [{'uri': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3375'}}]
pred_answer:
 [{'res': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q164561'}}]

id: 49
question: Which frequent flyer program has the most airlines?
ref sparql: PREFIX wdt: <http://www.wikidata.org/prop/direct/> PREFIX wd: <http://www.wikidata.org/entity/> PREFIX p: <http://www.wikidata.org/prop/> PREFIX ps: <http://www.wikidata.org/prop/statement/> SELECT ?frequentflyer WHERE { ?airlines wdt:P31 wd:Q46970 . ?airlines p:P4446/ps:P4446 ?frequentflyer . }  